In [1]:
import time

from google import genai

# For extracting vertex experiment details.
from google.cloud import aiplatform
from google.cloud.aiplatform.metadata import context
from google.cloud.aiplatform.metadata import utils as metadata_utils
from google.genai import types

# For data handling.
import jsonlines
import pandas as pd

# For visualization.
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# For evaluation metric computation.
from rouge_score import rouge_scorer
from tqdm import tqdm

# For fine tuning Gemini model.
import vertexai
import json 

In [ ]:
PROMPT_TEMPLATE = """
You are an expert in propaganda analysis. Analyze the following text. 
Always keep in mind that you are only looking for russian propaganda, not any other type of propaganda. Any other type of propaganda should be ignored and labeled as "No Propaganda".

Text:
"{text}"

1. Identify all Low level fine-grained labels that apply to the text. 
2. Identify the main low level fine-grained label that best describes the text.
3. Identify the high level label that best describes the text.

Low level fine-grained labels
🔴 1. Reductio ad Hitlerum / Nazi Analogies
Standard label: Reduction ad Hitlerum / Nazi Analogies
Includes: reductium ad hitlerum, reduction ad hitlerum, Second World War analogies, references to Hitler/SS, burning people like Nazis
🔴 2. Historical Distortion / Revisionism
Standard label: Historical Distortion / Revisionism
Includes: historical distortion, revisionism, cherry-picking historical events, WWII nostalgia, mirroring via historical analogies, historical debt, Soviet glorification, fake quotes attributed to leaders like Poroshenko
Includes: Cherry-Picking (choosing only facts that fit your narrative. Also known ad Card Stacking or Selective Omission: only information that supports a particular point of view is presented, while contradictory or unfavorable facts are deliberately omitted.
🔴 3. Dehumanization / Demonization
Standard label: Dehumanization / Demonization
Includes: demonization, dehumanization, calling others subhuman, Nazis / monsters / animals, emotional manipulation via atrocity claims
🔴 4. Deflect and Justify I: Gaslighting / Victimhood Framing / Victim Inversion (Primary function: Reverses moral polarity — the aggressor is portrayed as the victim.)
Key signals:
Standard label: Victimhood Framing / Inversion of Blame
Includes: victim inversion, victimhood framing, victim blaming, blame shifting, moral deflection, Hero-villain storytelling, perpetrator becomes the victim, Ukrainians are bombing themselves, doing genocide on their own people.
Includes: active reversal of good and evil
Includes: Denying facts by saying it was done by the other side.
Includes: Moral inversion
Includes: Gaslighting / Denial of Wrongdoing
🔴 5. Deflect and Justify II: Whataboutism (Primary function: Rationalizes or excuses actions by redirecting blame or reframing causes.)
Standard label: Deflect and Justify
Subtypes (can be annotated separately):
Blaming the other side in hypocrisy
False moral equivalence (Includes: "but the West invaded Iraq", "Saudi Arabia kills more people", "all sides commit war crimes”)
And what about Gaza
🔴 6. Accusation of Propaganda / Media Distrust
Standard label: Accusation of Propaganda / Media Distrust
Includes: the other side is propaganda, inversion, assertion that Western media lies, mockery of mainstream narratives, accusing others of brainwashing
Thought-Terminating Clichés if used to silence trust in institutions.
🔴 7. Conspiracy Narratives
Standard label: Conspiracy / Deep State Narratives
Includes: CIA / Nuland plots, biolabs, Zionist imperialism, puppet governments, secret funding, Western orchestration of coups, media control, assertion-as-proof
🔴 8. Guilt by Association
Standard label: Guilt by Association
Includes: Azov used = entire military is Nazi, "Zelensky is Jewish but surrounded by Nazis", past affiliations implying present guilt
🔴 9. Delegitimisation of opponent: Sarcasm / Ridicule / Mockery / Scarecrow / Straw man
Standard label: Sarcasm / Ridicule as Delegitimization
Includes: snark, mocking tone, "oh but Zelensky is Jewish so Nazis can’t exist, right?", eye-roll emoji language
Exaggerated or absurd version of opponent's position
Thought-Terminating Clichés: Many sarcastic or mocking phrases ("you believe the West? lol") function similarly.
🔴 10. Emotional Manipulation / Shock Appeal
Standard label: Emotional Manipulation / Shock Tactics
Includes: rape / torture / beheading narratives, children in basements, mass graves, gas chambers references
Appeal to fear/ nuclear threats
Economic or Religious Fearmongering
🔴 11. Appeal to Authority and False Authority / Fabrication (!might need merging/redestribution with 15)
Standard label: False Authority / Fabrication
Includes: fake quotes, fabricated court rulings, nonexistent statistics, unverified claims framed as fact
Includes Appeal to Authority
🔴 12. Us vs. Them Framing / Identity Dichotomy / Bandwagoning / Group Identity Appeal
Standard label: Dichotomy / Us vs. Them Framing
Includes: nationalist binaries, West vs. East, good patriots vs. evil globalists, pro-Russian people vs. “sheeple”
Includes: “you’re with us or you’re with the Nazis”, patriotic emoji spam, #IStandWithRussia, tribal solidarity
Includes: Bandwagon Appeals (Join the Crowd) - group identity appeal, tribal solidarity, and hashtags like #IStandWithRussia, which often function via bandwagoning.
Includes: Patriotic Appeal
🔴 13. Justification via Realpolitik, Moral relativism and Moral Detachment /Appeal to Tradition and nature of things
Standard label: Realpolitik Framing / Moral Detachment
Includes: cool analytical tone to justify invasion, "every country defends its sphere of influence", geopolitical realism
🔴 14. Triumphalism / Inevitability Framing
Standard label: Triumphalism / Victory Framing
Includes: Russia is winning, "when the dust settles, only the strong survive", "truth will prevail" type messages
🔴 15. Information Laundering / Plausible Deniability / Spreading Uncertainty
Standard label: Information Laundering / Rumor Seeding
Includes: “unconfirmed reports say…”, fake photo disclaimers, misleading but deniable headlines
Includes: doubt and rumor Seeding
🔴 16. Anti-Establishment / Anti-Globalist Positioning
Standard label: Anti-Establishment / Anti-Western Interventionism
Includes: criticism of NATO, EU, UN, WEF, neocolonial framing, blaming West for instability in Global South
🔴 17.  Logical Fallacy
Use this as a meta-tag to annotate false logic, especially when training for argument quality or rationality checks:
False dichotomy and Black and white fallacy (Oversimplifying complex situations into binary categories)
False causality (post hoc justification)
Causal Oversimplification
False Origin Attribution
Argument from Ignorance (Claims something is true because it hasn’t been disproven. "No one has proven the Biolabs don’t exist.”)
Circular Reasoning (Conclusion is restated as its own justification. "Russia is fighting Nazis because they are Nazis.”)
Appeal to Probability (Assumes that because something could happen, it will.)
False Balance (Gives equal weight to disproven and factual claims. Some say russia was invaded, some say it was provoked)
🔴 18.  No Propaganda
Use this as a tag if text does not contain any propaganda, can only appear alone.
High level 
🔶 0. No Propaganda 
Use this as a tag if 18. No propaganda is low level label
🔶 1. Distort Reality and Rewrite the Past
(Goal: Undermine truth, reshape history, and legitimize the present through distortion)
Subtypes:
2. Historical Distortion / Revisionism
11. False Authority / Fabrication
15. Information Laundering / Rumor Seeding
17. Logical Fallacy (especially False Origin Attribution, Circular Reasoning, Appeal to Ignorance)
🧠 Rationale: These tactics challenge fact-based narratives through denial, fabrication, or invented “evidence” — they are truth-subverting tools.
🔶 2. Shift Blame and Justify Aggression
(Goal: Reframe the aggressor as victim or rational actor; displace guilt)
Subtypes:
4. Deflect and Justify I: Victimhood Framing / Inversion of Blame
5. Deflect and Justify II: Whataboutism / Moral Relativism
13. Realpolitik Framing / Moral Detachment
17. Logical Fallacy (False causality, False balance)
🧠 Rationale: These techniques seek to invert the moral framework and rationalize or emotionally excuse wrongdoing, often by pointing fingers elsewhere.
🔶 3. Delegitimize the Opponent
(Goal: Undermine the credibility, morality, or coherence of the enemy)
Subtypes:
1. Reduction ad Hitlerum / Nazi Analogies
3. Dehumanization / Demonization
8. Guilt by Association
9. Sarcasm / Ridicule / Strawman
12. Us vs. Them Framing / Identity Dichotomy
6. Accusation of Propaganda / Media Distrust
🧠 Rationale: These techniques strip legitimacy from adversaries by turning them into villains, liars, extremists, or inhuman entities.
🔶 4. Manufacture Consent and Identity
(Goal: Rally support, polarize identity, and create in-group loyalty)
Subtypes:
12. Us vs. Them Framing / Identity Dichotomy
14. Triumphalism / Victory Framing
16. Anti-Establishment / Anti-Western Interventionism
10. Emotional Manipulation / Shock Tactics	
7. Conspiracy Narratives
🧠 Rationale: This group focuses on building loyalty, emotional investment, and polarized identity alignment — with both fear and pride as tools.
🔶 5. Confuse and Distract
(Goal: Overwhelm critical thinking by flooding discourse with noise, uncertainty, or implausible claims)
Subtypes:
7. Conspiracy Narratives
15. Information Laundering / Rumor Seeding
6. Accusation of Propaganda / Media Distrust
17. Logical Fallacy (Red herrings, Appeal to probability, False dichotomy)
🧠 Rationale: These tactics often don’t seek to convince, but to undermine clarity, trust, and consensus — enabling passivity or cynicism.

Now return the labels in a JSON format with the following structure:
{{"Low Level": ["label1", "label2", ...], "Main low level": "main_low_level_label", "High-level": "high_level_label"}}

Follow the json format strictly. Do not add any additional text or explanations. Just use the integers for labeling, no strings.
The high-level labels dont have to be exclusive to their respective low-level labels. If another high-level label fits better, use it. 
Remember to always return Integers only!
"""

In [3]:
def generate_from_pickle():
    # Define the path to your pickle file and an output JSON file
    input_pickle_file = r"C:\Users\david\PY\golden_test_set_fixed.pkl"
    output_json_path = "gemini_results_2.0-flash-001_baseline.json"

    # --- Step 1: Load the data from the pickle file ---
    try:
        df = pd.read_pickle(input_pickle_file)
        print(f"✅ Loaded pickle file with {len(df)} rows.")
    except FileNotFoundError:
        print(f"❌ Error: Pickle file not found at '{input_pickle_file}'")
        return
    except Exception as e:
        print(f"❌ Error loading pickle file: {e}")
        return

    # Initialize the client for your fine-tuned model
    try:
        client = genai.Client(
            vertexai=True,
            project="472207425752",
            location="us-central1",
        )
        model = "projects/472207425752/locations/us-central1/endpoints/1130470576679288832"
        #model= "gemini-2.0-flash-001"
        print("✅ Gemini client and model path initialized successfully.")
    except Exception as e:
        print(f"❌ Error initializing Gemini client: {e}")
        return

    # Your fixed response schema, formatted as a JSON object
    response_schema_json = {
        "type": "OBJECT",
        "properties": {
            "Low Level": {"type": "ARRAY", "items": {"type": "INTEGER", "minimum": 0}},
            "Main low level": {"type": "INTEGER", "minimum": 0},
            "High-level": {"type": "INTEGER", "minimum": 0}
        },
        "required": ["Low Level", "Main low level", "High-level"]
    }
    
    # List to store all results
    processed_data = []

    # --- Step 2: Loop through each row of the DataFrame ---
    for i, row in df.iterrows():
        text_to_analyze = str(row.get("text", "")).strip()
        record_id = row.get("id", f"index-{i}")
        
        print(f"\n--- Processing record {i + 1} (ID: {record_id}) ---")

        # Skip empty text entries to avoid API errors
        if not text_to_analyze:
            print("⚠️ Skipping entry with empty text.")
            processed_data.append({
                "id": record_id,
                "text": "",
                "analysis_result": None,
                "error": "Empty input text"
            })
            continue

        analysis_result = None
        error_message = None

        # --- Step 3: Run the model with the text and handle potential errors ---
        try:
            # Format the prompt with the text from the current row
            full_prompt = PROMPT_TEMPLATE.format(text=text_to_analyze)
            
            # Create the content for the API call with the full prompt
            text_part = types.Part.from_text(text=full_prompt)
            contents = [
                types.Content(
                    role="user",
                    parts=[text_part]
                )
            ]

            # Generate content with the specified configuration
            generate_content_config = types.GenerateContentConfig(
                temperature=0,
                top_p=1,
                seed=0,
                max_output_tokens=8192,
                safety_settings=[
                    types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
                    types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
                    types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
                    types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF")
                ],
                response_mime_type="application/json",
                response_schema=response_schema_json,
            )

            # Stream the response and reconstruct the full output
            full_response_text = ""
            for chunk in client.models.generate_content_stream(
                model = model,
                contents=contents,
                config=generate_content_config,
            ):
                full_response_text += chunk.text

            # Parse the JSON response
            analysis_result = json.loads(full_response_text)
            print("✅ Successfully analyzed record. Result:")
            print(json.dumps(analysis_result, indent=4))
        
        except json.JSONDecodeError as e:
            error_message = f"JSON parsing failed: {e}. Raw output: {full_response_text}"
            print(f"❌ Error for record {i + 1}: {error_message}")
        except Exception as e:
            error_message = f"API call failed: {e}"
            print(f"❌ Unexpected error for record {i + 1}: {error_message}")

        # --- Step 4: Store the result and error message ---
        record = {
            "id": record_id,
            "text": text_to_analyze,
            "analysis_result": analysis_result,
            "error": error_message
        }
        processed_data.append(record)

    # --- Step 5: Save the final results to a JSON file ---
    try:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(processed_data, f, ensure_ascii=False, indent=4)
        print(f"\n✅ All results saved to '{output_json_path}'")
    except Exception as e:
        print(f"❌ Error saving JSON file: {e}")

# Run the main function
generate_from_pickle()

✅ Loaded pickle file with 187 rows.
✅ Gemini client and model path initialized successfully.

--- Processing record 1 (ID: index-0) ---
✅ Successfully analyzed record. Result:
{
    "Low Level": [
        1,
        15
    ],
    "Main low level": 1,
    "High-level": 3
}

--- Processing record 2 (ID: index-1) ---
✅ Successfully analyzed record. Result:
{
    "Low Level": [
        4,
        6,
        7,
        12,
        16
    ],
    "Main low level": 4,
    "High-level": 2
}

--- Processing record 3 (ID: index-2) ---
✅ Successfully analyzed record. Result:
{
    "Low Level": [
        1,
        3,
        4,
        10,
        12
    ],
    "Main low level": 4,
    "High-level": 2
}

--- Processing record 4 (ID: index-3) ---
✅ Successfully analyzed record. Result:
{
    "Low Level": [
        1,
        3,
        7,
        8,
        12,
        16
    ],
    "Main low level": 7,
    "High-level": 5
}

--- Processing record 5 (ID: index-4) ---
✅ Successfully analyzed record.

KeyboardInterrupt: 